## Remove bg

In [23]:
import os
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim

import numpy as np
from PIL import Image
import glob

from data_loader import RescaleT
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET # full size version 173.6 MB
from model import U2NETP # small version u2net 4.7 MB

In [24]:
# normalize the predicted SOD probability map
def normPRED(d):
    ma = torch.max(d)
    mi = torch.min(d)

    dn = (d-mi)/(ma-mi)

    return dn

def mask_output(image_name,pred,d_dir):

    predict = pred
    predict = predict.squeeze()
    predict_np = predict.cpu().data.numpy()

    im = Image.fromarray(predict_np*255).convert('RGB')
    img_name = image_name.split(os.sep)[-1]
    image = io.imread(image_name)
    imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

    pb_np = np.array(imo)

    aaa = img_name.split(".")
    bbb = aaa[0:-1]
    imidx = bbb[0]
    for i in range(1,len(bbb)):
        imidx = imidx + "." + bbb[i]

    # imo.save(d_dir+imidx+'.png')
    return imo

In [25]:
# --------- 1. get image path and name ---------
model_name = 'u2net'#u2netp
input_path = 'input_images'
output_path = 'result_images'

# image_dir = os.path.join(os.getcwd(), 'test_data', 'test_images')
# prediction_dir = os.path.join(os.getcwd(), 'test_data', model_name + '_results' + os.sep)
image_dir = os.path.join(os.getcwd(), input_path)
prediction_dir = os.path.join(os.getcwd(), output_path)
model_dir = os.path.join(os.getcwd(), 'saved_models', model_name, model_name + '.pth')

img_name_list = glob.glob(image_dir + os.sep + '*')
print(img_name_list)

# --------- 2. dataloader ---------
#1. dataloader
test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                    lbl_name_list = [],
                                    transform=transforms.Compose([RescaleT(320),
                                                                    ToTensorLab(flag=0)])
                                    )
test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                    batch_size=1,
                                    shuffle=False,
                                    num_workers=1)

# --------- 3. model define ---------
if(model_name=='u2net'):
    print("...load U2NET---173.6 MB")
    net = U2NET(3,1)
elif(model_name=='u2netp'):
    print("...load U2NEP---4.7 MB")
    net = U2NETP(3,1)

if torch.cuda.is_available():
    net.load_state_dict(torch.load(model_dir))
    net.cuda()
else:
    net.load_state_dict(torch.load(model_dir, map_location='cpu'))
net.eval()

# --------- 4. inference for each image ---------
for i_test, data_test in enumerate(test_salobj_dataloader):

    print("inferencing:",img_name_list[i_test].split(os.sep)[-1])

    inputs_test = data_test['image']
    inputs_test = inputs_test.type(torch.FloatTensor)

    if torch.cuda.is_available():
        inputs_test = Variable(inputs_test.cuda())
    else:
        inputs_test = Variable(inputs_test)

    d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

    # normalization
    pred = d1[:,0,:,:]
    pred = normPRED(pred)

    # save results to test_results folder
    if not os.path.exists(prediction_dir):
        os.makedirs(prediction_dir, exist_ok=True)
    mask = mask_output(img_name_list[i_test],pred,prediction_dir)

    # generate remove bg from original image & mask
    img = Image.open(os.path.join(input_path, img_name_list[i_test].split(os.sep)[-1]))
    w, h = img.size
    img_empty = Image.new("RGBA", (w, h), (0, 0, 0, 0))
    mask = mask.convert('L').resize(img.size)
    out = Image.composite(img, img_empty, mask)
    out.save(os.path.join(output_path, img_name_list[i_test].split(os.sep)[-1].split('.')[0] + '.png'))

    del d1,d2,d3,d4,d5,d6,d7

['d:\\U-2-Net\\input_images\\F_03.png']
...load U2NET---173.6 MB
inferencing: F_03.png


## Remove bg by mask

In [27]:
import os
import cv2
import numpy as np
from PIL import Image

In [28]:
im_path = 'test_data/test_images/0002-01.jpg' # original
mask_path = 'test_data/u2net_results/0002-01.png' # mask
save_path = 'result_images/'

In [29]:
img = Image.open(im_path)
w, h = img.size
img_empty = Image.new("RGBA", (w, h), (0, 0, 0, 0))
mask = Image.open(mask_path).convert('L').resize(img.size)
out = Image.composite(img, img_empty, mask)
out.save(save_path + 'out.png')